In [1]:
import docx
import pandas as pd

In [2]:
gt = pd.read_excel("./train data/train_data_markup.xlsx", keep_default_na=False)
gt

,Number,Name,Differences,Description,Complience Level
0,8800,Receiving Call Notifications,SSTS misses the following content:\n The outpu...,HMX description:\nThe driver can accept or rej...,LC
1,31523,Adding Internet Radio to Favorites List,UC misses the following content:\nUsers can se...,Users can add or remove songs from favorites a...,LC
2,25957,Mute/unmute the FM Radio playback,There is no soft button icon and description i...,The user clicks on the radio and clicks on the...,PC
3,26160,Filtering the internet radio stations,ssts hasn't info about this,-,NA
4,11467,Revoke access to the vehicle from a driver or ...,-,The driver opens the ATOM application on his i...,FC
5,30371,ERA Self-diagnosis,SSTS starts Every time the ignition switch is ...,Self-diagnosis includes the following items:\n...,LC
6,26771,Turn on and off hotspot via VA,SSTS misses the following content:\n VA will i...,Users can turn on or off the vehicle hotspot a...,LC
7,28561,Setting Hotspot name & password,UC misses the following content: \n This funct...,Users can modify the name and password of the ...,LC
8,8604,Source selection for Media output,No online&BLT music HMI button guide.\nThis us...,Support USB/SD/online/BLT media type\nAtom car...,NC
9,6583,Driver initiate a call through SWP,SSTS describes that only when the vehicle is s...,Additionally SSTS describes:\n- a numeric keyb...,NC


In [3]:
dataset = []
for idx in gt.index:
    line = gt.loc[idx]
    if line["Complience Level"] == "NA":
        continue
        
    uc_doc = docx.Document(f"./train data/HMI/UC-{line['Number']}.docx")
    uc_text = "\n".join(paragraph.text for paragraph in uc_doc.paragraphs[1:])
    ssts_doc = docx.Document(f"./train data/SSTS/SSTS-{line['Number']}.docx")
    ssts_text = "\n".join(paragraph.text for paragraph in ssts_doc.paragraphs[1:])
    
    name = uc_doc.paragraphs[0].text.split('\xa0')[-1].strip()
    assert name.lower() == line["Name"].lower()
    
    for task in ("Differences", "Description", "Complience Level"):
        features = {
            "task": task,
            "name": name,
            "uc_text": uc_text,
            "ssts_text": ssts_text,
            "answer": line[task]
        }
        dataset.append(features)
dataset = pd.DataFrame(dataset)
dataset.head(3)

,task,name,uc_text,ssts_text,answer
0,Differences,Receiving Call Notifications,Preconditions\nEntertainment system is ready. ...,Functional Description:\nWhen a mobile phone c...,SSTS misses the following content:\n The outpu...
1,Description,Receiving Call Notifications,Preconditions\nEntertainment system is ready. ...,Functional Description:\nWhen a mobile phone c...,HMX description:\nThe driver can accept or rej...
2,Complience Level,Receiving Call Notifications,Preconditions\nEntertainment system is ready. ...,Functional Description:\nWhen a mobile phone c...,LC


In [4]:
dataset.to_csv("./train data/dataset.csv", index=False)